In [1]:
import numpy as np

In [22]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i, j] > 0:
                    # calculate error
                    piqj = np.dot(P[i, :], Q[:, j])
                    eij = R[i, j] - piqj

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i, k] += alpha * (2 * eij * Q[k, j] - beta * P[i, k])
                        Q[k, j] += alpha * (2 * eij * P[i, k] - beta * Q[k, j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i, j] > 0:
                    e += pow(R[i, j] - piqj, 2)
                    for k in range(K):
                        e += (beta/2) * (pow(P[i, k],2) + pow(Q[k, j],2))
        # 0.001: local minimum
        if e < 0.001:
            break
    return P, Q.T

In [29]:
%%timeit
R = np.array([
    [5,3,0,1],
    [4,0,0,1],
    [1,1,0,5],
    [1,0,0,4],
    [0,1,5,4],
    [2,1,3,0]])
N, M = R.shape
K = 3
P = np.random.rand(N,K)
Q = np.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)

457 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
nP, nQ = matrix_factorization(R, P, Q, K)
nP

array([[ 2.40186054e+00,  7.27135739e-01,  1.73977380e-01],
       [ 1.39380900e+00,  1.28776232e+00, -4.45114366e-01],
       [ 9.14493425e-04,  1.26623132e+00,  1.81550627e+00],
       [ 2.17578776e-02,  1.13269862e+00,  1.34874005e+00],
       [ 5.15449802e-01,  1.87853702e+00,  7.15550435e-01],
       [ 6.35062789e-01,  8.80223043e-01,  4.42870066e-01]])

In [20]:
def original(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [27]:
%%timeit
nP, nQ = original(R, P, Q, K)

634 ms ± 3.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
nP, nQ = original(R, P, Q, K)
nP

array([[ 2.40243082,  0.72801996,  0.18200415],
       [ 1.37626423,  1.29378661, -0.45193298],
       [-0.00438887,  1.26144937,  1.80945567],
       [ 0.02050678,  1.12273064,  1.34940388],
       [ 0.52969098,  1.88726221,  0.69953208],
       [ 0.63220029,  0.87694603,  0.45375773]])